# Tutorial 2: Charged Systems: Counterion Condensation

## Table of contents
* [Introduction](#Introduction)
* [System setup](#System-setup)
* [First runs and system cleanup](#First-runs-and-system-cleanup)
* [Production run and analysis](#Production-run-and-analysis)
* [Overcharging by added salt](#Overcharging-by-added-salt)



## Introduction

In this tutorial, we simulate a charged system consisting of a fixed charged rod with ions around it. This setup represents a simplified model for polyelectrolyte gels. We will investigate the condensation of ions onto the oppositely charged rod and compare the results to a meanfield analytical solution obtained from Poisson-Boltzmann (PB) theory.
Finally we will go beyond the expected applicability of PB and add concentrated additional salt ions to observe an overcharging effect.

The tutorial follows "Deserno, Markus, Christian Holm, and Sylvio May. "Fraction of condensed counterions around a charged rod: Comparison of Poisson− Boltzmann theory and computer simulations. Macromolecules 33.1 (2000): 199-206, [10.1021/ma990897o](https://doi.org/10.1021/ma990897o)". We refer to that publication for further reading.

In [ ]:
from espressomd import System, interactions, electrostatics

import numpy as np
from scipy import optimize
%matplotlib inline
import matplotlib.pyplot as plt

np.random.seed(41)

# System setup

After importing the necessary **ESPResSo** features and external modules, we define the cubical system geometry and some physical parameters (which define our unit system).

In [ ]:
# system parameters
rod_length = 50
bjerrum_length = 1.0

# we assume a unit system where the elementary charge and the thermal energy are both 1
system = System(box_l=3*[rod_length])
kT = 1.
q_e = 1.

system.time_step = 0.01
system.cell_system.skin = 0.4

We will build the charged rod from individual particles that are fixed in space. With this, we can use the particle-based electrostatics methods of **ESPResSo**. For analysis, we give the rod particles a different type than the counterions.

In [ ]:
# interaction parameters
wca_epsilon = 1.0
ion_diameter = 1.0
rod_radius = 1.0
# particle types 
rod_type = 1
counterion_type = 2

**Exercise:**

* Setup the purely repulsive Weeks-Chandler-Anderson (WCA) interaction ([Non-bonded Interactions](http://espressomd.org/html/doc/inter_non-bonded.html)) between the ions and between the ions and the rod particles. Use the parameters introduced in the cell above.

**Hints:**
* The WCA potential uses the same parameters as the Lennard-Jones potential, but the cutoff and shift are calculated automatically
* Use the Lorentz combining rule (arithmetic mean) to determine the ``sigma`` parameter of the interaction between the rod particles and the ions

```python
#ion-ion interaction
system.non_bonded_inter[counterion_type,counterion_type].wca.set_params(
      epsilon=wca_epsilon, sigma=ion_diameter)

# ion-rod interaction
system.non_bonded_inter[counterion_type,rod_type].wca.set_params(
      epsilon=wca_epsilon, sigma=ion_diameter/2. + rod_radius)
```

Now we need to place the particles in the box

**Exercise:**
* Implement a function to place the rod particles along the $x_3$ axis in the middle of the simulation box and the ions randomly distributed 
* Use the signature ``setup_rod_and_counterions(system, ion_valency, counterion_type, rod_charge_dens, N_rod_beads, rod_type) ``
* determine the number of counterions from the condition of neutrality for the whole system (the rod should be positive, the counterions negative)
* Assign the rod particles and cointerions their correct ``type``
* Give the counterions a charge ``q`` according to their ``ion_valency``
* Give the rod particles a charge such that the ``rod_charge_dens`` is uniformely distributed along the ``N_rod_beads`` individual particles
* Fix the rod particles in space so they do not get moved if forces act upon them

**Hints:**
* Look into [Eespresso particle properties](http://espressomd.org/html/doc/espressomd.html?#module-espressomd.particle_data) to find the keywords to set charges and fix particles
* use np.random.random() to generate the counterion positions

```python
def setup_rod_and_counterions(system, ion_valency, counterion_type,
                 rod_charge_dens, N_rod_beads, rod_type):

    # calculate charge of the single rod beads
    rod_length = system.box_l[2]
    total_rod_charge = rod_charge_dens*rod_length
    rod_charge_per_bead = total_rod_charge/N_rod_beads

    # number of counterions
    N_ions = int(total_rod_charge/ion_valency)
    
    rod_zs = np.linspace(0,rod_length,num = N_rod_beads,endpoint = False)
    rod_positions = np.column_stack(([system.box_l[0]/2.]*N_rod_beads, 
                                    [system.box_l[1]/2.]*N_rod_beads,
                                     rod_zs))
    
    system.part.add(pos = rod_positions,type=[rod_type]*N_rod_beads, 
                    q=[rod_charge_per_bead]*N_rod_beads, 
                    fix=[3*[True]]*N_rod_beads)
    
    ion_positions = np.random.random((N_ions,3)) * system.box_l
   
    counter_ions = system.part.add(pos = ion_positions,type=[counterion_type]*N_ions, q=[-ion_valency]*N_ions)

    return counter_ions
```

In [ ]:
counterion_valency = 1
rod_charge_dens = 2

# number of beads that make up the rod
N_rod_beads = 50

setup_rod_and_counterions(system, counterion_valency, counterion_type,
             rod_charge_dens, N_rod_beads, rod_type)

#check that the particle setup was done correctly
assert(abs(sum(system.part[:].q))<1e-10)
assert(np.all(system.part.select(type=rod_type).fix))

Now we set up the electrostatics method to calculate the forces and energies from the longrange coulomb interaction. **ESPResSo** uses so-called <tt>actors</tt> for electrostatics, magnetostatics and hydrodynamics. This ensures that unphysical combinations of algorithms are avoided, for example simultaneous usage of two electrostatic interactions. Adding an actor to the system also activates the method and calls necessary initialization routines. Here, we define a P$^3$M object using the Bjerrum length and rms force error. This automatically starts a tuning function which tries to find optimal parameters for P$^3$M and prints them to the screen. For more details, see the [Espresso documentation](http://espressomd.org/html/doc/electrostatics.html).

In [ ]:
p3m_params = {'prefactor':kT*bjerrum_length*q_e**2,
              'accuracy':1e-3}

For the accuracy, **ESPResSo** estimates the relative error in the force calculation introduced by the approximations of $P^3M$. We choose a relatively poor accuracy (large value) for this tutorial to make it run faster. For your own production simulations you should reduce that number.

**Exercise:**
* Set up a ``p3m`` instance and add it to the ``actors`` of the system

```python
p3m = electrostatics.P3M(**p3m_params)
system.actors.add(p3m)
```

Before we can start the simulation, we need to remove the overlap between particles to avoid large forces which would crash the simulation. For this, we use the steepest descent integrator with a relative convergence criterium for forces and energies.

In [ ]:
def remove_overlap(system, sd_params):   
    # Removes overlap by steepest descent until forces or energies converge
    # Set up steepest descent integration
    system.integrator.set_steepest_descent(f_max=0,
                                           gamma=sd_params['damping'],
                                           max_displacement=sd_params['max_displacement'])
    
    # Initialize integrator to obtain initial forces
    system.integrator.run(0)
    maxforce = np.max(np.linalg.norm(system.part[:].f, axis = 1))
    energy = system.analysis.energy()['total']
    
    i = 0
    while i < sd_params['max_steps']//sd_params['emstep']:
        prev_maxforce = maxforce
        prev_energy = energy
        print(prev_energy)
        system.integrator.run(sd_params['emstep'])
        maxforce = np.max(np.linalg.norm(system.part[:].f, axis = 1))
        relforce = np.abs((maxforce-prev_maxforce)/prev_maxforce)
        energy = system.analysis.energy()['total']
        relener = np.abs((energy-prev_energy)/prev_energy)
        print("minimization step: {:4.0f}\tmax. rel. force change:{:+3.3e}\trel. energy change:{:+3.3e}".format((i+1)*sd_params['emstep'],relforce, relener))
        if relforce < sd_params['f_tol'] or relener < sd_params['e_tol']:
            break
        i += 1
        
    system.integrator.set_vv()

In [ ]:
steepest_descent_params = {'f_tol':1e-2,
                          'e_tol':1e-5,
                          'damping':30,
                          'max_steps':10000,
                          'max_displacement':0.01,
                          'emstep':10}

remove_overlap(system,steepest_descent_params)

After the overlap is removed, we activate a thermostat to simulate the system at a given temperature.

In [ ]:
langevin_params = {'kT':kT,
                   'gamma':0.5,
                   'seed':42}
system.thermostat.set_langevin(**langevin_params)

## First runs and system cleanup

Now we are ready to implement the integration loop. As we are interested in the condensation of counterions on the rod, the physical quantity of interest is the density $\rho(r)$ of charges around the rod, where $r$ is the distance from the rod. We need many samples (frames) to calculate the density from histograms.
The samples should be taken from an equilibrated system and not be correlated to each other.

**Exercise:**

* Write a function to integrate the system and return the total energy and the radial distance of the ions from the rod
* Use the signature ``integrate_calc_observables(system, N_frames, steps_per_frame, ion_types)``
* ``N_frames`` is the total number of samples taken from the system
* ``steps_per_frame`` is the the number of integration steps between consecutive frames
* ``ion_types`` is a list of types for which the radial distances should be calculated. For the moment we only have counterions, but later we will also add additional salt ions for which we would like to calculate the density
* return a list of the total system energy as well as a dictionary of the radial positions sorted by ion type, e.g. ``radial_distances[counterion_type] = [1.1,4.5,2.7,...]``

**Hints:**
* We are only interested in the distance from the rod, so the $x_3$ coordinate is irrelevant
* Use ``system.part.select(type=...)`` to get only the particles of a specific type
* The system energy can be calculated using **ESPReSso**'s [analysis module](http://espressomd.org/html/doc/espressomd.html?#module-espressomd.analyze), the radial distance has to be computed manually
* Particles will likely move through different periodic images of the system. Use the folded positions for distance calculations

```python
def integrate_calc_observables(system, N_frames, steps_per_frame, ion_types):
    energies = []
    radial_distances = []

    particles_by_type = {}
    radial_distances = {}
    for ion_type in ion_types:
        particles_by_type[ion_type] = system.part.select(type=ion_type)
        radial_distances[ion_type] = []

    system_center = np.array(system.box_l)/2.

    for frame_idx in range(N_frames):

        print(f'progress={frame_idx/N_frames*100:.0f}%', end='\r')

        system.integrator.run(steps_per_frame)

        energies.append(system.analysis.energy()['total'])

        for ion_type, ions in particles_by_type.items():
            for ion in ions:
                radial_distances[ion_type].append(np.linalg.norm(ion.pos_folded[0:2]-system_center[0:2]))
    print('progress=100%')
    return energies, radial_distances
```



Before running the simulations for the histograms, we need to decide how long we need to equilibrate the system. For this we plot the total energy vs the time steps.

In [ ]:
# number of samples taken
N_frames = 500

# number of timesteps per frame
steps_per_frame = 20
energies_first_run, _ = integrate_calc_observables(system, N_frames, 
                                         steps_per_frame, [counterion_type])

In [ ]:
#plot time in time_steps so we can judge the number of warmup steps and the steps per frame for uncorrelated data
ts = np.linspace(0,N_frames*steps_per_frame, num = N_frames)
fig1 = plt.figure()
plt.plot(ts, energies_first_run)
plt.xlabel('time steps')
plt.ylabel('system total energy')

In [ ]:
warmup_steps = 5000
steps_per_frame = 100

To run the simulation with different parameters, we need a way to reset the system and return it to an empty state before setting it up again.

**Exercise:**
* write a function ``clear_system(system)`` that
    * turns off the thermostat
    * removes all particles
    * resets the system clock

**Hints:**
* the relevant parts of the documentation can be found here: [Thermostats](http://espressomd.org/html/doc/system_setup.html#thermostats), [Particle List](http://espressomd.org/html/doc/espressomd.html#espressomd.particle_data.ParticleList), [System properties](http://espressomd.org/html/doc/espressomd.html#module-espressomd.system)

```python
def clear_system(system):
    system.thermostat.turn_off()
    system.part.clear()
    system.time = 0.
```

In [ ]:
clear_system(system)

## Production run and analysis
Now we are finally ready to run the simulations and produce the data we can compare to the Poisson-Boltzmann predictions. First we define the parameters and then loop over them.

In [ ]:
runs = [{'params':{'counterion_valency':2, 'rod_charge_dens':1},
         'distances':None},
        {'params':{'counterion_valency':1, 'rod_charge_dens':2},
         'distances':None}
       ]
N_frames = 1000

**Exercise:**
* run the simulation for the parameters given in the cell above and save the radial distances in the corresponding dictionary for analysis

**Hints:**
* Don't forget to clear the system once one parameter set is finished and redo the complete setup
* Don't forget to ``tune()`` the ``p3m`` instance after each change of parameters. If we reuse the p3m that was tuned before, likely the desired accuracy will not be achieved. 

```python
clear_system(system)
for run in runs:
    setup_rod_and_counterions(system, run['params']['counterion_valency'], counterion_type,
                 run['params']['rod_charge_dens'], N_rod_beads, rod_type)
    p3m.tune()
    remove_overlap(system, steepest_descent_params)
    system.thermostat.set_langevin(**langevin_params)
    system.integrator.run(warmup_steps)
    print('setup and warmup done, starting simulation')
    energies, distances = integrate_calc_observables(system, N_frames, 
                                       steps_per_frame, [counterion_type])
    clear_system(system)
    run['distances'] = distances[counterion_type]
    print('simulation for parameters {} done\n'.format(run['params']))
```

**Question**
* Why does the second simulation take much longer than the first one?

The rod charge density is doubled, so the total charge of the counterions needs to be doubled, too. Since their valency is only half of the one in the first run, there will be four times more counterions in the second run

We plot the density of counterions around the rod as the normalized integrated radial counterion charge distribution function $P(r)$, meaning the integrated probability to find an amount of charge within the radius $r$. We express the rod charge density $\lambda$ in terms of the dimensionless Manning parameter $\xi = \lambda l_B / e$ where $l_B$ is the Bjerrum length and $e$ the elementary charge

In [ ]:
def calc_cum_hist(values, bins, normalize = True):
    hist, _ = np.histogram(values, bins=log_bins)
    cum_hist = np.cumsum(hist)
    if normalize:
        cum_hist = cum_hist/cum_hist[-1]
    return cum_hist

In [ ]:
r_min = rod_radius+ion_diameter/2.
r_max = system.box_l[0]/2.
log_bins=np.logspace(np.log10(r_min), np.log10(r_max),num=30)

fig, ax = plt.subplots()
for run in runs:
    cum_hist = calc_cum_hist(run['distances'], log_bins)
    manning_xi = run['params']['rod_charge_dens'] * bjerrum_length / q_e
    ax.plot(log_bins[1:],cum_hist, label = r'$\xi ={}, \nu = {}$'.format(manning_xi, 
                                                                 run['params']['counterion_valency']))
    
ax.set_xscale('log')
ax.legend()
plt.xlabel('r')
plt.ylabel('P(r)')

In the semilogarithmic plot we see an inflection point of the cumulative charge distribution which is the indicator for ion condensation. To compare to the meanfield approach of PB, we calculate the solution of the analytical expressions given in [10.1021/ma990897o](https://doi.org/10.1021/ma990897o)

In [ ]:
def eq_to_solve_for_gamma(gamma, manning_parameter, rod_radius, max_radius):
    #eq 7 - eq 6 from 10.1021/ma990897o
    return gamma*np.log(max_radius/rod_radius) - np.arctan(1/gamma) + np.arctan((1-manning_parameter)/gamma)

def calc_manning_radius(gamma,max_radius):
    #eq 7 from 10.1021/ma990897o
    return max_radius*np.exp(-np.arctan(1./gamma)/gamma)  

def calc_PB_probability(r, manning_parameter, gamma, manning_radius):
    #eq 8 and 9 from 10.1021/ma990897o
    return 1./manning_parameter + gamma/manning_parameter * np.tan(gamma*np.log(r/manning_radius))


For multivalent counterions, the manning parameter $\xi$ has to be multiplied by the valency $\nu$. The result depends only on the product of ``rod_charge_dens`` and ``ion_velancy``, so we only need one curve

In [ ]:
rod_charge_density = runs[0]['params']['rod_charge_dens']
ion_valency = runs[0]['params']['counterion_valency']
manning_parameter_times_valency = bjerrum_length*rod_charge_density*ion_valency

gamma = optimize.fsolve(eq_to_solve_for_gamma, 1, args = (manning_parameter_times_valency,r_min,r_max))
manning_radius = calc_manning_radius(gamma, r_max)

PB_probability = calc_PB_probability(log_bins, manning_parameter_times_valency,gamma, manning_radius)

ax.plot(log_bins, PB_probability, label = r'PB $\xi \cdot \nu$ = {}'.format(manning_parameter_times_valency))
ax.legend()
ax.set_xscale('log')
fig

We see a good correspondence of the PB solution with the simulation results in the case where the counterion valence is 1, but there are stronger deviations if the counterions are more charged. 

Poisson Boltzmann makes two simplifying assumptions: Particles are points and there are no correlations between the particles. Both is not given in the simulation. Excluded volume effects can only lower the density, but we see in the figure that the simulated density is always larger that the calculated one. This means that correlation effects cause the discrepancy

## Overcharging by added salt

Above simulations were performed for a system where all ions come from dissociation off the polyelectrolyte. We can also investigate systems where there are additional salt ions present.

In [ ]:
def add_salt(system, anion_params, cation_params):  
    
    N_anions = anion_params['number']
    N_cations = cation_params['number']
    
    anion_positions = np.random.random((N_anions,3)) * system.box_l
    cation_positions = np.random.random((N_cations,3)) * system.box_l
   
   
    anions = system.part.add(pos = anion_positions,type=[anion_params['type']]*N_anions, 
                    q=[-anion_params['valency']]*N_anions)
    cations = system.part.add(pos = cation_positions,type=[cation_params['type']]*N_cations, 
                    q=[cation_params['valency']]*N_cations) 
    
    return anions, cations

In [ ]:
anion_params = {'type':3,
                'valency':2,
                'number':500}
cation_params = {'type':4,
                 'valency':2,
                 'number':500}
rod_length = 21

steps_per_frame_salt = 20
N_frames_salt = 1000

counterion_valency = 1 
rod_charge_dens = 1

all_ion_types = [counterion_type, anion_params['type'],cation_params['type'] ]

In [ ]:
#set interactions of salt with the rod and all ions
for salt_type in [anion_params['type'], cation_params['type']]:
    system.non_bonded_inter[salt_type,rod_type].wca.set_params(
          epsilon=wca_epsilon, sigma=ion_diameter/2. + rod_radius)
    for ion_type in all_ion_types:
        system.non_bonded_inter[salt_type,ion_type].wca.set_params(
          epsilon=wca_epsilon, sigma=ion_diameter)

In [ ]:
clear_system(system)
system.box_l = 3*[rod_length]
setup_rod_and_counterions(system, counterion_valency, counterion_type,
                   rod_charge_dens, N_rod_beads, rod_type)
anions, cations = add_salt(system, anion_params, cation_params)
assert( abs(sum(anions.q)+sum(cations.q))<1e-10 )
p3m.tune()
remove_overlap(system, steepest_descent_params)
system.thermostat.set_langevin(**langevin_params)
system.integrator.run(warmup_steps)
print('setup and warmup done, starting simulation')
energies, distances = integrate_calc_observables(system, N_frames_salt, 
                                   steps_per_frame_salt, all_ion_types)
print('simulation done')


**Exercise:**
* Create the cumulative charge histogram normalized by the negative of the rod charge



**Hints**
* create cumulative histograms for each type of ion in the system
* Add them together weighted by their valency to get the cumulative charge distribution
* use ``calc_cum_hist(...,normalize = False)`` to account for the fact that there are a different number of ions depending on their type

```python
counterion_hist = calc_cum_hist(distances[counterion_type], log_bins, normalize = False)
anion_hist = calc_cum_hist(distances[anion_params['type']], log_bins, normalize = False)
cation_hist = calc_cum_hist(distances[cation_params['type']], log_bins, normalize = False)

charge_hist = cation_params['valency'] * cation_hist - anion_params['valency'] * anion_hist - counterion_valency * counterion_hist
charge_hist = charge_hist/charge_hist[-1]
```

In [ ]:
r_max = system.box_l[0]/2.
log_bins=np.logspace(np.log10(r_min), 
                     np.log10(r_max),
                     num=30)
fig2, ax2 = plt.subplots()
ax2.plot(log_bins[1:], charge_hist)
ax2.set_xscale('log')
plt.xlabel('r')
plt.ylabel('P(r)')

Sampling is insufficient to adequately resolve the bigger distances,but you should observe a pronounced overcharging effect at short distances.